In [ ]:
!pip -q install pandas beautifulsoup4 lxml requests spacy tqdm
!python -m spacy download ru_core_news_sm -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 84.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [ ]:
URL = "https://ir.yandex.ru/financial-releases?year=2025&report=q3"

In [ ]:
html = requests.get(URL, headers={"User-Agent":"Mozilla/5.0"}).text
soup = BeautifulSoup(html, "lxml")

In [ ]:
tables = pd.read_html(html, header=None, decimal=",", thousands=" ")

/tmp/ipython-input-3699163378.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html, header=None, decimal=",", thousands=" ")


In [ ]:
def score_table(df: pd.DataFrame) -> int:
    text = " ".join(map(str, df.head(10).values.flatten())).lower()
    keys = ["выручка", "скорректирован", "ebitda", "рентабельность", "чистая прибыль", "девять месяцев", "три месяца"]
    return sum(k in text for k in keys)

scored = [(i, score_table(tables[i])) for i in range(len(tables))]
scored_sorted = sorted(scored, key=lambda x: x[1], reverse=True)
scored_sorted[:10]


[(0, 7),
 (1, 6),
 (2, 6),
 (5, 6),
 (4, 5),
 (6, 5),
 (7, 5),
 (8, 5),
 (14, 4),
 (3, 3)]

In [ ]:
best_idx = scored_sorted[0][0]
df_raw = tables[best_idx].copy()
df_raw.head(20)

,0,1,2,3,4,5,6,7
0,В млрд руб.,NaN,"Три месяца, закончившиеся 30 сентября","Три месяца, закончившиеся 30 сентября","Три месяца, закончившиеся 30 сентября","Девять месяцев, закончившиеся 30 сентября","Девять месяцев, закончившиеся 30 сентября","Девять месяцев, закончившиеся 30 сентября"
1,NaN,NaN,2025,2024,Изменение,2025,2024,Изменение
2,Результаты Группы Яндекса,Выручка,366.1,276.8,32%,1005.1,754.4,33%
3,NaN,Скорректированный показатель EBITDA,78.1,54.7,43%,193.0,139.9,38%
4,NaN,Рентабельность скорректированного показателя E...,"21,3%","19,8%","1,5 п.п.","19,2%","18,5%","0,7 п.п."
5,NaN,Скорректированная чистая прибыль,44.7,25.1,78%,87.9,69.6,26%
6,Поисковые сервисы и ИИ,Выручка,138.5,127.0,9%,388.3,343.2,13%
7,NaN,Скорректированный показатель EBITDA,63.0,58.0,9%,176.3,151.1,17%
8,NaN,Рентабельность скорректированного показателя E...,"45,5%","45,7%","-0,2 п.п.","45,4%","44,0%","1,4 п.п."
9,Городские сервисы,Выручка,200.4,146.9,36%,561.2,403.6,39%


In [ ]:
def normalize_value(x):
    if pd.isna(x):
        return None

    s = str(x).strip()
    s = s.replace("\xa0", " ").replace("−", "-") # NBSP, минус
    s_low = s.lower()

    if s_low in {"—", "-", "н/прим", "n/a", "na"}:
        return None

    # (4,2) преобразует в -4,2
    is_neg = bool(re.match(r"^\(.*\)$", s))
    s = s.strip("()")

    # убирает пробелы в числах: "1 005,1" в "1005,1"
    s = re.sub(r"(?<=\d)\s+(?=\d)", "", s)

    # 21,3% в 21,3
    s = s.replace("%", "")

    # запятую в точку
    s = s.replace(",", ".")

    # оставляет только число/точку/минус
    if not re.search(r"\d", s):
        return None
    s = re.sub(r"[^0-9\.\-]", "", s)

    try:
        val = float(s)
        return -val if is_neg else val
    except:
        return None

In [ ]:
def flatten_columns(df: pd.DataFrame) -> pd.DataFrame:
    if isinstance(df.columns, pd.MultiIndex):
        df = df.copy()
        df.columns = [
            " ".join([str(c).strip() for c in col if str(c).strip() != "nan"]).strip()
            for col in df.columns.values
        ]
    else:
        df = df.copy()
        df.columns = [str(c).strip() for c in df.columns]
    return df

df = flatten_columns(df_raw)
df.head()


,0,1,2,3,4,5,6,7
0,В млрд руб.,NaN,"Три месяца, закончившиеся 30 сентября","Три месяца, закончившиеся 30 сентября","Три месяца, закончившиеся 30 сентября","Девять месяцев, закончившиеся 30 сентября","Девять месяцев, закончившиеся 30 сентября","Девять месяцев, закончившиеся 30 сентября"
1,NaN,NaN,2025,2024,Изменение,2025,2024,Изменение
2,Результаты Группы Яндекса,Выручка,366.1,276.8,32%,1005.1,754.4,33%
3,NaN,Скорректированный показатель EBITDA,78.1,54.7,43%,193.0,139.9,38%
4,NaN,Рентабельность скорректированного показателя E...,"21,3%","19,8%","1,5 п.п.","19,2%","18,5%","0,7 п.п."


In [ ]:
text_cols = [c for c in df.columns if df[c].astype(str).str.contains("Выручка|EBITDA|Рентабельность|чист", case=False, na=False).any()]

In [ ]:
df2 = df.copy()

In [ ]:
def is_mostly_numeric(col):
    sample = df2[col].head(30).astype(str)
    hits = sample.str.contains(r"\d", regex=True, na=False).mean()
    return hits > 0.6

non_num_cols = [c for c in df2.columns[:4] if not is_mostly_numeric(c)]
non_num_cols

['0', '1']

In [ ]:
id_cols = non_num_cols[:2] if len(non_num_cols) >= 2 else [df2.columns[0]]
value_cols = [c for c in df2.columns if c not in id_cols]

facts = df2.melt(id_vars=id_cols, value_vars=value_cols, var_name="col", value_name="value_raw")
facts["value"] = facts["value_raw"].apply(normalize_value)

facts.head(15)

,0,1,col,value_raw,value
0,В млрд руб.,NaN,2,"Три месяца, закончившиеся 30 сентября",0.3
1,NaN,NaN,2,2025,2025.0
2,Результаты Группы Яндекса,Выручка,2,366.1,366.1
3,NaN,Скорректированный показатель EBITDA,2,78.1,78.1
4,NaN,Рентабельность скорректированного показателя E...,2,"21,3%",21.3
5,NaN,Скорректированная чистая прибыль,2,44.7,44.7
6,Поисковые сервисы и ИИ,Выручка,2,138.5,138.5
7,NaN,Скорректированный показатель EBITDA,2,63.0,63.0
8,NaN,Рентабельность скорректированного показателя E...,2,"45,5%",45.5
9,Городские сервисы,Выручка,2,200.4,200.4


In [ ]:
if len(id_cols) == 2:
    facts = facts.rename(columns={id_cols[0]:"entity", id_cols[1]:"metric"})
else:
    facts = facts.rename(columns={id_cols[0]:"entity_metric"})
    facts["entity"] = facts["entity_metric"].astype(str).str.replace(r"\s{2,}", " ", regex=True).str.strip()
    facts["metric"] = None

# период и тип колонки: 2025/2024/Изменение и "3 мес / 9 мес" пробую вытащить по тексту колонки
def parse_period(col_name: str):
    s = str(col_name)
    year = re.search(r"(20\d{2})", s)
    yr = year.group(1) if year else None
    is_change = "Измен" in s or "Change" in s
    return yr, is_change

facts[["year","is_change"]] = facts["col"].apply(lambda c: pd.Series(parse_period(c)))
facts["source"] = f"table_{best_idx}"

# фильтр: только строки где есть число или процент/изменение
facts_clean = facts.dropna(subset=["value_raw"]).copy()

# выбрасываю пустые
facts_clean = facts_clean[~facts_clean["value_raw"].astype(str).str.strip().isin(["", "nan"])]

facts_clean.head(20)


,entity,metric,col,value_raw,value,year,is_change,source
0,В млрд руб.,NaN,2,"Три месяца, закончившиеся 30 сентября",0.3,None,False,table_0
1,NaN,NaN,2,2025,2025.0,None,False,table_0
2,Результаты Группы Яндекса,Выручка,2,366.1,366.1,None,False,table_0
3,NaN,Скорректированный показатель EBITDA,2,78.1,78.1,None,False,table_0
4,NaN,Рентабельность скорректированного показателя E...,2,"21,3%",21.3,None,False,table_0
5,NaN,Скорректированная чистая прибыль,2,44.7,44.7,None,False,table_0
6,Поисковые сервисы и ИИ,Выручка,2,138.5,138.5,None,False,table_0
7,NaN,Скорректированный показатель EBITDA,2,63.0,63.0,None,False,table_0
8,NaN,Рентабельность скорректированного показателя E...,2,"45,5%",45.5,None,False,table_0
9,Городские сервисы,Выручка,2,200.4,200.4,None,False,table_0


In [ ]:
import spacy
nlp = spacy.load("ru_core_news_sm")

In [ ]:
main_text = soup.get_text("\n", strip=True)

# lines = [ln.strip() for ln in main_text.split("\n") if ln.strip()]
# bullet_lines = [ln for ln in lines if ln.startswith("•") or ln.startswith("*")]

li_texts = [li.get_text(" ", strip=True) for li in soup.select("li")]
bullet_lines = [t for t in li_texts if re.search(r"\d", t)]
bullet_lines[:10], len(bullet_lines)

(['Выручка за третий квартал составила 366,1 млрд рублей, увеличившись на\n                32% год к году.',
  'Скорректированный показатель EBITDA за третий квартал составил 78,1 млрд\n                рублей или 21,3% от выручки, рост 1,5 п.п. по сравнению с аналогичным\n                периодом прошлого года.',
  'Рекламная выручка Яндекса выросла на 11% до 116 млрд рублей за третий\n                квартал и на 15% до 325 млрд рублей за девять месяцев 2025 года по\n                отношению к аналогичным периодам 2024 года.',
  'Подписная выручка Яндекса выросла на 48% до 23 млрд рублей за третий\n                квартал и на 49% до 65 млрд рублей за девять месяцев 2025 года по\n                отношению к аналогичным периодам 2024 года.',
  'Яндекс до конца 2027 года запустит в российских городах 20 тысяч\n                роботов-доставщиков. Новые роботы впервые будут производиться серийно, а\n                доставка ими будет обходиться дешевле, чем курьерами.',
  '9 сентября 20

In [ ]:
METRICS = [
    "выручка", "рекламная выручка", "подписная выручка",
    "скорректированный показатель ebitda", "ebitda",
    "рентабельность", "скорректированная чистая прибыль", "чистая прибыль",
]

metric_re = re.compile("|".join([re.escape(m) for m in sorted(METRICS, key=len, reverse=True)]), re.I)

# "366,1 млрд рублей", "80 рублей на акцию"
money_re = re.compile(r"(\(?-?\d[\d\s]*[.,]?\d*\)?)\s*(млрд|млн|тыс)?\s*(руб(лей|ля|\.|)|₽)?", re.I)

def extract_facts_from_line(line: str):
    line_clean = line.lstrip("•* ").strip()
    doc = nlp(line_clean)

    # entity: ORG проверка
    orgs = [ent.text for ent in doc.ents if ent.label_ == "ORG"]
    entity = orgs[0] if orgs else "Яндекс"

    metric_m = metric_re.search(line_clean)
    metric = metric_m.group(0) if metric_m else None

    m = money_re.search(line_clean)
    value_raw = m.group(0).strip() if m else None
    value = normalize_value(m.group(1)) if m else None

    # период: "за третий квартал" / "за девять месяцев"
    period = None
    if re.search(r"трет(ий|ьего)\s+квартал", line_clean, re.I):
        period = "Q3 2025"
    if re.search(r"девят(ь|и)\s+месяц", line_clean, re.I):
        period = "9M 2025"

    return {
        "entity": entity,
        "metric": metric,
        "period": period,
        "value_raw": value_raw,
        "value": value,
        "source": "text_bullets",
        "context": line_clean
    }

text_facts = [extract_facts_from_line(ln) for ln in bullet_lines]
text_facts_df = pd.DataFrame([f for f in text_facts if f["metric"] and f["value_raw"]])

text_facts_df

,entity,metric,period,value_raw,value,source,context
0,Яндекс,Выручка,Q3 2025,"366,1 млрд рублей",366.1,text_bullets,"Выручка за третий квартал составила 366,1 млрд..."
1,Яндекс,Скорректированный показатель EBITDA,Q3 2025,"78,1 млрд\n рублей",78.1,text_bullets,Скорректированный показатель EBITDA за третий ...
2,Яндекс,Рекламная выручка,9M 2025,11,11.0,text_bullets,Рекламная выручка Яндекса выросла на 11% до 11...
3,Яндекс,Подписная выручка,9M 2025,48,48.0,text_bullets,Подписная выручка Яндекса выросла на 48% до 23...
4,Электронной коммерции,Рекламная выручка,None,"15,5\n млрд рублей",15.5,text_bullets,Рекламная выручка Городских сервисов в третьем...
5,Yandex Cloud,выручка,9M 2025,2025,2025.0,text_bullets,За девять месяцев 2025 года выручка ИИ-сервисо...
6,Yandex Cloud,Выручка,None,2025,2025.0,text_bullets,В третьем квартале 2025 года сервисами безопас...
7,Яндекс 360 24,выручка,None,360 24,36024.0,text_bullets,Сервисами Яндекс 360 24 пользуются более 160 т...


In [ ]:
table_facts = facts_clean.copy()
table_facts["period"] = table_facts["col"].astype(str)
table_facts["context"] = None

final_df = pd.concat(
    [
        table_facts[["entity","metric","period","value_raw","value","source"]],
        text_facts_df[["entity","metric","period","value_raw","value","source"]],
    ],
    ignore_index=True
)

final_df.head(30)


,entity,metric,period,value_raw,value,source
0,В млрд руб.,NaN,2,"Три месяца, закончившиеся 30 сентября",0.3,table_0
1,NaN,NaN,2,2025,2025.0,table_0
2,Результаты Группы Яндекса,Выручка,2,366.1,366.1,table_0
3,NaN,Скорректированный показатель EBITDA,2,78.1,78.1,table_0
4,NaN,Рентабельность скорректированного показателя E...,2,"21,3%",21.3,table_0
5,NaN,Скорректированная чистая прибыль,2,44.7,44.7,table_0
6,Поисковые сервисы и ИИ,Выручка,2,138.5,138.5,table_0
7,NaN,Скорректированный показатель EBITDA,2,63.0,63.0,table_0
8,NaN,Рентабельность скорректированного показателя E...,2,"45,5%",45.5,table_0
9,Городские сервисы,Выручка,2,200.4,200.4,table_0


In [ ]:
final_df.tail(30)

,entity,metric,period,value_raw,value,source
122,Результаты Группы Яндекса,Выручка,7,33%,33.0,table_0
123,NaN,Скорректированный показатель EBITDA,7,38%,38.0,table_0
124,NaN,Рентабельность скорректированного показателя E...,7,"0,7 п.п.",NaN,table_0
125,NaN,Скорректированная чистая прибыль,7,26%,26.0,table_0
126,Поисковые сервисы и ИИ,Выручка,7,13%,13.0,table_0
127,NaN,Скорректированный показатель EBITDA,7,17%,17.0,table_0
128,NaN,Рентабельность скорректированного показателя E...,7,"1,4 п.п.",NaN,table_0
129,Городские сервисы,Выручка,7,39%,39.0,table_0
130,NaN,Скорректированный показатель EBITDA,7,152%,152.0,table_0
131,NaN,Рентабельность скорректированного показателя E...,7,"3,2 п.п.",NaN,table_0
